In [10]:
import os

project_path = r"C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project"

os.chdir(project_path)
print(f"Moved to: {os.getcwd()}")


Moved to: C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project


In [11]:
%pwd

'C:\\Users\\user\\anaconda3\\envs\\Chicken-disease-Classification-Project'

In [17]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [18]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import  read_yaml, create_directories # Assumed utility file

In [19]:
#configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAM_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config['prepare_callbacks']  # dictionary-style access

        model_ckpt_dir = os.path.dirname(config['checkpoint_model_filepath'])

        create_directories([
            Path(model_ckpt_dir),
            Path(config['tensorboard_root_log_dir'])
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config['root_dir']),
            tensorboard_root_log_dir=Path(config['tensorboard_root_log_dir']),
            checkpoint_model_filepath=Path(config['checkpoint_model_filepath'])
        )

        return prepare_callbacks_config


In [ ]:
#components
import os
from datetime import datetime
import tensorflow as tf
from pathlib import Path
import time


class PrepCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self) -> tf.keras.callbacks.TensorBoard:
        """Creates a TensorBoard callback with timestamped log directory."""
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        tb_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_log_dir)

    @property
    def _create_ckpt_callbacks(self) -> tf.keras.callbacks.ModelCheckpoint:
        """Creates a ModelCheckpoint callback."""
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self) -> list:
        """Returns a list of TensorBoard and ModelCheckpoint callbacks."""
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

       

In [22]:
#pipeline
import logging

logger = logging.getLogger(__name__)

try:
    logger.info("Starting configuration manager...")
    config = ConfigurationManager()

    logger.info("Fetching prepare callbacks configuration...")
    prepare_callbacks_config = config.get_prepare_callback_config()

    logger.info("Initializing PrepCallbacks...")
    prepare_callbacks = PrepCallbacks(config=prepare_callbacks_config)

    logger.info("Getting TensorBoard and checkpoint callbacks list...")
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    logger.info("Pipeline setup completed successfully.")

except Exception as e:
    logger.exception("An error occurred during the pipeline setup.")
    raise


[2025-05-30 11:50:42,684: INFO: 4275068883: Starting configuration manager...]
[2025-05-30 11:50:42,693: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-05-30 11:50:42,725: INFO: common: YAML file: params.yaml loaded successfully]
[2025-05-30 11:50:42,753: INFO: common: Created directory at: artifacts]
[2025-05-30 11:50:42,760: INFO: 4275068883: Fetching prepare callbacks configuration...]
[2025-05-30 11:50:42,774: INFO: common: Created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-05-30 11:50:42,777: INFO: common: Created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-05-30 11:50:42,803: INFO: 4275068883: Initializing PrepCallbacks...]
[2025-05-30 11:50:42,816: INFO: 4275068883: Getting TensorBoard and checkpoint callbacks list...]
[2025-05-30 11:50:42,822: INFO: 4275068883: Pipeline setup completed successfully.]
